#  FrozenLake
Today you are going to learn how to survive walking over the (virtual) frozen lake through discrete optimization.

<img src="http://vignette2.wikia.nocookie.net/riseoftheguardians/images/4/4c/Jack's_little_sister_on_the_ice.jpg/revision/latest?cb=20141218030206" alt="a random image to attract attention" style="width: 400px;"/>


In [1]:
import gym

#create a single game instance
env = gym.make("FrozenLake-v0")

#start new game
env.reset();

[2017-01-04 11:23:30,082] Making new env: FrozenLake-v0


In [2]:
# display the game state
env.render()

SFFF
FHFH
FFFH
HFFG



### legend

![img](https://cdn-images-1.medium.com/max/800/1*MCjDzR-wfMMkS0rPqXSmKw.png)

### Gym interface

The three main methods of an environment are
* __reset()__ - reset environment to initial state, _return first observation_
* __render()__ - show current environment state (a more colorful version :) )
* __step(a)__ - commit action __a__ and return (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the MDP has just finished, False if still in progress
 * _info_ - some auxilary stuff about what just happened. Ignore it for now

In [3]:
print ("initial observation code:", env.reset())
print ('printing observation:')
env.render()
print ("observations:",env.observation_space, 'n=',env.observation_space.n)
print ("actions:",env.action_space, 'n=',env.action_space.n)

initial observation code: 0
printing observation:
SFFF
FHFH
FFFH
HFFG

observations: Discrete(16) n= 16
actions: Discrete(4) n= 4


In [4]:
print ("taking action 2 (right)")
new_obs, reward, is_done, _ = env.step(2)
print ("new observation code:",new_obs)
print ("reward:", reward)
print ("is game over?:",is_done)
print ("printing new state:")
env.render()

taking action 2 (right)
new observation code: 1
reward: 0.0
is game over?: False
printing new state:
SFFF
FHFH
FFFH
HFFG
  (Right)


In [5]:
action_to_i = {
    'left':0,
    'down':1,
    'right':2,
    'up':3
}

### Play with it
* Try walking 5 steps without falling to the (H)ole
 * Bonus quest - get to the (G)oal
* Sometimes your actions will not be executed properly due to slipping over ice
* If you fall, call __env.reset()__ to restart

In [6]:
env.step(action_to_i['up'])
env.render()

SFFF
FHFH
FFFH
HFFG
  (Up)


### Policy

* The environment has a 4x4 grid of states (16 total), they are indexed from 0 to 15
* From each states there are 4 actions (left,down,right,up), indexed from 0 to 3

We need to define agent's policy of picking actions given states. Since we have only 16 disttinct states and 4 actions, we can just store the action for each state in an array.

This basically means that any array of 16 integers from 0 to 3 makes a policy.

In [8]:
import numpy as np
def get_random_policy():
    """
    Build a numpy array representing agent policy.
    This array must have one element per each of 16 environment states.
    Element must be an integer from 0 to 3, representing action
    to take from that state.
    """
    return np.random.randint(0, 4, 16)

In [9]:
np.random.seed(1234)
policies = [get_random_policy() for i in range(10**4)]

assert all([len(p) == 16 for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == 3, 'maximal action id should be 3'
action_probas = np.unique(policies,return_counts=True)[-1] /10**4. /16.
print ("Action frequencies over 10^4 samples:",action_probas)
assert np.allclose(action_probas,[0.25]*4,atol=0.05), "The policies aren't uniformly random (maybe it's just an extremely bad luck)"
print ("Seems fine!")

Action frequencies over 10^4 samples: [ 0.25014375  0.25130625  0.2495375   0.2490125 ]
Seems fine!


### Let's evaluate!
* Implement a simple function that runs one game and returns the total reward

In [212]:
def sample_reward(env,policy,t_max=25):
    """
    Interact with an environment, return sum of all rewards.
    If game doesn't end on t_max (e.g. agent walks into a wall), 
    force end the game and return whatever reward you got so far.
    """
    s = env.reset()
    total_reward = 0
    
    for t in range(t_max):
        s,r,done,_ = env.step(policy[s])
        total_reward += r
        if done : break
    
    return total_reward

In [165]:
print ("generating 10^3 sessions...")
rewards = [sample_reward(env,get_random_policy()) for _ in range(10**3)]
assert all([type(r) in (int,float) for r in rewards]), 'sample_reward must return a single number'
assert all([0 <= r <= 1 for r in rewards]), 'total rewards should be between 0 and 1 for frozenlake'
print ("Looks good!")

generating 10^3 sessions...


AssertionError: total rewards should be between 0 and 1 for frozenlake

In [166]:
def evaluate(policy,n_times=100):
    """Run several evaluations and average the score the policy gets."""
    rewards = [sample_reward(env,policy) for i in range(n_times)]
    return float(np.mean(rewards))
        

In [167]:
def print_policy(policy):
    """a function that displays a policy in a human-readable way"""
    lake = "SFFFFHFHFFFHHFFG"
    
    # where to move from each tile
    arrows = ['<v>^'[a] for a in policy]
    
    #draw arrows above S and F only
    signs = [arrow if tile in "SF" else tile for arrow,tile in zip(arrows,lake)]
    
    for i in range(0,16,4):
        print (' '.join(signs[i:i+4]))

print ("random policy:")
print_policy(get_random_policy())

random policy:
v ^ > ^
> H ^ H
^ > v H
H > v G


### Random search

In [168]:
best_policy = None
best_score = -float('inf')

from tqdm import tqdm
for i in tqdm(range(10000)):
    policy = get_random_policy()
    score = evaluate(policy)
    if score > best_score:
        best_score = score
        best_policy = policy
        print ("New best score:",score)
        print ("Best policy:")
        print_policy(best_policy)

  0%|          | 8/10000 [00:00<04:47, 34.81it/s]

New best score: -0.17399
Best policy:
> ^ ^ >
> H > H
> ^ v H
H ^ ^ G
New best score: -0.06645000000000001
Best policy:
> < ^ ^
^ H < H
< ^ < H
H > < G


  0%|          | 21/10000 [00:00<04:23, 37.89it/s]

New best score: -0.05946
Best policy:
> v ^ >
> H < H
< ^ < H
H < ^ G
New best score: -0.05682
Best policy:
v < ^ >
^ H v H
^ ^ > H
H < v G
New best score: -0.052920000000000016
Best policy:
< > < <
v H > H
> > ^ H
H > ^ G


  0%|          | 33/10000 [00:00<05:09, 32.21it/s]

New best score: -0.05097
Best policy:
v v v v
v H v H
v ^ > H
H > < G
New best score: -0.02797000000000001
Best policy:
> v > >
> H v H
v ^ < H
H < > G


  1%|          | 58/10000 [00:01<05:19, 31.09it/s]

New best score: -0.003260000000000003
Best policy:
v v v >
v H v H
^ v v H
H ^ v G


  1%|          | 74/10000 [00:02<05:12, 31.71it/s]


KeyboardInterrupt: 

### Genetic algorithm

In [169]:
def crossover(policy1,policy2,p=0.5):
    """
    for each state, with probability p take action from policy1, else policy2
    """
    policy = []
    for i in range(len(policy1)):
        policy.append(np.random.choice([policy1[i], policy2[i]], p=[p, 1-p]))
    
    return policy

In [170]:
np.random.seed(1234)
policies = [crossover(get_random_policy(),get_random_policy()) 
            for i in range(10**4)]

assert all([len(p) == 16 for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == 3, 'maximal action id should be 3'
print ("Seems fine!")

Seems fine!


In [209]:

n_epochs = 200 #how many cycles to make
pool_size = 200 #how many policies to maintain
n_crossovers = 50 #how many crossovers to make on each step
n_mutations = 20 #how many mutations to make on each tick


In [210]:
print ("initializing...")
pool = []
for i in range(pool_size):
    pool.append(get_random_policy())
pool_scores = []#<evaluate every policy in the pool, return list of scores>
for p in pool:
    pool_scores.append(evaluate(p))

initializing...


In [198]:
assert type(pool) == type(pool_scores) == list
assert len(pool) == len(pool_scores) == pool_size
assert all([type(score) in (float,int) for score in pool_scores])


In [199]:
def rand_from_top(array, Ntop):
    N = len(array)
    top = []
    
    for i in range(Ntop) :
        top.append(array.index(max(array)))
        array[array.index(max(array))] = 0
        
    return np.random.choice(top)
        
            

In [236]:
#main loop
for epoch in range(n_epochs):
    print ("Epoch %s:"%epoch)
    
    crossovered = []#<crossover random guys from pool, n_crossovers total>
    for i in range(n_crossovers):
        n1 = rand_from_top(pool_scores, 50)
        n2 = rand_from_top(pool_scores, 50)
        crossovered.append(crossover(pool[n1], pool[n2], p=0.5))
        
    mutated = []#<add several new policies at random, n_mutations total>
    for i in range(n_mutations):
        n = rand_from_top(pool_scores, 50)
        mutated.append(crossover(pool[n], get_random_policy(), p=0.8))
    
    assert type(crossovered) == type(mutated) == list
    
    #evaluate new scores
    new_policies = crossovered + mutated
    
    #add new sessions to the pool
    pool = pool + new_policies
    pool_scores = []
    for p in pool:
        pool_scores.append(evaluate(p))
    
    #select pool_size best policies
    selected_indices = np.argsort(pool_scores)[-pool_size:]
    pool = [pool[i] for i in selected_indices]
    pool_scores = [pool_scores[i] for i in selected_indices]
    
    best = np.argmax(pool_scores)
    
    #print the best policy so far (last in ascending score order)
    print ("best score:",pool_scores[best])
    print_policy(pool[best])

Epoch 0:
best score: 0.4
< < < >
< H < H
^ v < H
H > v G
Epoch 1:
best score: 0.4
< v v >
< H < H
^ v < H
H > v G
Epoch 2:
best score: 0.42
< < < >
< H < H
^ v < H
H > v G
Epoch 3:
best score: 0.41
< < < >
< H < H
^ v < H
H > v G
Epoch 4:
best score: 0.41
< < < >
< H < H
^ v < H
H > v G
Epoch 5:
best score: 0.42
< < < >
< H < H
^ v < H
H > v G
Epoch 6:
best score: 0.4
< < < >
< H < H
^ v < H
H > v G
Epoch 7:
best score: 0.38
< < < >
< H < H
^ v < H
H > v G
Epoch 8:
best score: 0.41
< < < >
< H < H
^ v < H
H > v G
Epoch 9:
best score: 0.43
< < < >
< H < H
^ v < H
H > v G
Epoch 10:


KeyboardInterrupt: 

## moar

The parameters of the genetic algorithm aren't optimal, try to find something better. (size, crossovers and mutations)

Try alternative crossover and mutation strategies
* prioritize crossover over higher-scores
* only mutate several random actions from existing policy, not the whole.
* try to select a more diverse pool, not just best scorers

See which combination works best!